# Using dataclasses and dataclasses-json to keep track of application-wide settings


Volker Hilsenstein, June 2019


### Rationale: dealing with many  application-wide settings

In an application, one often has a number of parameters that are used in different functions or class methods. Now one is faced with the choice on how to store these parameters and pass them around. There seem to be several common approaches to this (the following list is not comprehensive):

* make each function or class method accept the required settings as argument. This leads to long parameter lists. Often several functions have the same or similar parameters. Functions that call other functions often have to accepts lots of parameters that they only use to pass them on to the functions they call. This quickly leads to very large function declarations and unwieldy code.
* some approaches to address one are for functions to accept unspecified named arguments using the `**` syntax that then simply get passed on to the functions they call. This is for example quite common in matplotlib, where lots of plotting parameters can be passed in as optional named arguments. There is a major drawback with this approach: documentation. When looking at a function `func_a(a, b=1, **optinonal_params)`  it is often not clear what named arguments `**optional_params` do without looking at all the functions that are called by `func_a` to which the `**optional_params` are passed onto. In terms of documentation this requires either duplicating
the documentation or referring to all the called functions.
* one often finds settings dictionaries that get passed to functions. The dictionary gets passed around. This can reduce a large number of parameters to pass around to a single (or a few) dictionarie/s. Again, this requires diligent documentation: what are the expected dictionary keys. What data types should be assigned to each key. Type hints and type checking are not well supported with this approach. Settings dictionaries can be easily serialized into a JSON or XML for reading/saving to disk. 
* encapsulating settings as class variables are another approach. In this case a class is used primarily as a namespace to hold all the functions (methods) and the parameters are stored in class instance variables that are accessible to all methods. This works, but introduces classes when they may not be needed. Also, it is nice to have fairly generic availability available as a simple function which does not require instantiating a class. 

In this notebook I explore the use of python dataclasses (part of the python standard library as of 3.7, a backport to 3.6 can be installed via `pip install dataclasses` to create a sinlge object that holds all the application settings. This is very similar to the approach of using a settings dictionary, but has the following advantes:

* **type hints** are required and **type checking** is supported. A linter can check whether a setting exists, which it cannot do for keys of a dictionary. These features make it easier to spot mistakes. 
* **documentation**: if a specific dataclass is passed to a function it is clear where the documentation belongs, it goes into the definition of the dataclass. Using a modern IDE such as VS Code or PyCharm one can quickly follow to the data class definition.
* **serialization**: by installing [`dataclasses-json`](https://pypi.org/project/dataclasses-json/)one can quickly serialize and de-serialize a settings object to a JSON string and save/load the settings from a configuration file.

# Example:

The example below is from an image analysis workflow (not reproduced here) in which glomeruli and podocytes within the glomeruli are segmented. The workflow consists of a number of different image analyis steps and some parameters (like typical volume sizes in voxels, probability thresholds) are required at several of the steps.

We define a dataclass that holds all of these settings. We then only need to keep track of the one 
dataclass instance during the workflow. 

Note the `@dataclass` and `@dataclass_json` decorators and the very concise syntax.

In [3]:
from dataclasses import dataclass
from dataclasses_json import dataclass_json

@dataclass_json
@dataclass
class ProcessSettings:
    """
    This docstring would be the single place to document these
    parameters
    
    e.g.
    
    glom_min_volume: (integer) The minimum number of voxels for an object to be considered
                               a glomerule
    and so on ...
    
    """
    # ranges to filter by size
    glom_min_vol: int = 30000
    glom_max_vol: int = 10000000
    podo_min_vol: int = 100
    podo_max_vol: int = 20000
    # threshold values
    threshold_adjust: float = 1.0 # thresh. adj. for flom crop
    thresh_podo_prob: float = 0.7
    thresh_bg_prob: float = 0.6
    thresh_h_val: float = 0.5
    # how far to expand the glomeruli when including podocytes
    glom_expand = 20
    # raw data channel assignments
    podo_ch: int = 1
    glom_ch: int = 0
    # ilastik classifier class labels
    podo_class: int = 1
    bg_class: int = 0

# Instantiate a settings object with default parameters

In [4]:
settings = ProcessSettings()

In [5]:
settings

ProcessSettings(glom_min_vol=30000, glom_max_vol=10000000, podo_min_vol=100, podo_max_vol=20000, threshold_adjust=1.0, thresh_podo_prob=0.7, thresh_bg_prob=0.6, thresh_h_val=0.5, podo_ch=1, glom_ch=0, podo_class=1, bg_class=0)

In [6]:
settings.thresh_bg_prob=0.7

In [7]:
settings

ProcessSettings(glom_min_vol=30000, glom_max_vol=10000000, podo_min_vol=100, podo_max_vol=20000, threshold_adjust=1.0, thresh_podo_prob=0.7, thresh_bg_prob=0.7, thresh_h_val=0.5, podo_ch=1, glom_ch=0, podo_class=1, bg_class=0)

# Instantiate a settings object with non-default values

Note that you get auto-completion support within Jupyter (and many other IDEs). You would not have this if you stored your settings in a dictionary. 

In [8]:
another_settings = ProcessSettings(podo_min_vol=10, podo_max_vol=1000)

In [9]:
another_settings

ProcessSettings(glom_min_vol=30000, glom_max_vol=10000000, podo_min_vol=10, podo_max_vol=1000, threshold_adjust=1.0, thresh_podo_prob=0.7, thresh_bg_prob=0.6, thresh_h_val=0.5, podo_ch=1, glom_ch=0, podo_class=1, bg_class=0)

# Serialize settings to a JSON string

this functionality is provided by the `@dataclass_json` decorator 

In [11]:
jsonstring = another_settings.to_json()
jsonstring

'{"glom_min_vol": 30000, "glom_max_vol": 10000000, "podo_min_vol": 10, "podo_max_vol": 1000, "threshold_adjust": 1.0, "thresh_podo_prob": 0.7, "thresh_bg_prob": 0.6, "thresh_h_val": 0.5, "podo_ch": 1, "glom_ch": 0, "podo_class": 1, "bg_class": 0}'

# Restore a settings object from a JSON string

In [13]:
ps = ProcessSettings.from_json(jsonstring)
ps

ProcessSettings(glom_min_vol=30000, glom_max_vol=10000000, podo_min_vol=10, podo_max_vol=1000, threshold_adjust=1.0, thresh_podo_prob=0.7, thresh_bg_prob=0.6, thresh_h_val=0.5, podo_ch=1, glom_ch=0, podo_class=1, bg_class=0)

# Outlook: GUI elements

It should not be too difficult to automatically create a minimalist GUI for a settings object defined in the above way. The type hints are there and the GUI element could be selected
automatically for each type.
If one uses simple float and int types, these would probably have to be text fields as there is
no hint about a suitable range for sliders.
However, one could address this by creating `float_range` and `int_range` types. However, using an existing solution such as `traitlets` may be the better approach.